In [8]:
def conv(a,b):
    tmp1=a[0]*b[0]+a[1]*b[1]
    tmp2=a[0]*b[1]+a[1]*b[0]
    return tmp1,tmp2

print(conv((0.2,0.8),(0.64,0.04)))
print(conv((0.04,0.64),(0.64,0.04)))
print(conv((0.04,0.64),(0.2,0.8)))

(0.16, 0.52)
(0.0512, 0.4112)
(0.52, 0.16)


In [9]:
0.2*0.64+0.8*0.04

0.16

In [10]:
def dot(a,b):
    return a[0]*b[0],a[1]*b[1]

def dot3(a,b,c):
    return a[0]*b[0]*c[0],a[1]*b[1]*c[1]

print(dot((0.2,0.8),(0.16,0.52)))

print(dot((0.2,0.8),(0.032,0.416)))

print(dot3((0.2,0.8),(0.2,0.8),(0.16,0.52)))

print(dot((0.0512,0.4112),(0.2,0.8)))

print(dot3((0.8,0.2),(0.8,0.2),(0.52,0.16)))

print(dot((0.64,0.04),(0.8,0.2)))

(0.032, 0.41600000000000004)
(0.0064, 0.3328)
(0.006400000000000001, 0.3328000000000001)
(0.01024, 0.32896000000000003)
(0.3328000000000001, 0.006400000000000001)
(0.512, 0.008)


In [11]:
import numpy as np
import random

def process_check_node(mu):
    def conv(a,b):
        tmp1=a[0]*b[0]+a[1]*b[1]
        tmp2=a[0]*b[1]+a[1]*b[0]
        return np.array([tmp1,tmp2])
    v_pass=dict()
    for i in range(4):
        filtered_mu = {key: val for key, val in mu.items() if key[0] == i}
        # print(filtered_mu)
        for k,v in filtered_mu.items():
            conv_target_dic=filtered_mu.copy()
            conv_target_dic.pop(k)
            # print(f"target{conv_target_dic}")

            v_pass[k] =conv_target_dic.pop(random.choice(list(conv_target_dic.keys())))
            for _,conv_target in conv_target_dic.items():
                v_pass[k] = conv(v_pass[k],conv_target)
            # print(f"v_pass{k}={v_pass[k]}")
    return v_pass

def process_judge(v_factor,v_pass,H):
    kai=np.zeros((6,2))
    for j in range(6):
        filtered_vpass = {key: val for key, val in v_pass.items() if key[1] == j}.copy()
        # print(filtered_vpass)
        dot_all=v_factor[j].copy()
        for v in filtered_vpass.values():
            dot_all=dot_all*v
        kai[j] = dot_all
    
    if np.all(kai[:,0]!=kai[:,1]):
        print("No equal")
        x_pred=(kai[:,0]<kai[:,1]).astype(int)
        if np.all(np.dot(H,x_pred) == np.array([0,0,0,0])):
            print("x_pred is correct")
            print(f"x_pred={x_pred}")
    return kai

def process_variable_node(v_factor,v_pass):
    mu=dict()
    for j in range(6):
        filtered_vpass = {key: val for key, val in v_pass.items() if key[1] == j}.copy()
        # print(f"filtered_vpass{filtered_vpass}")
        for k,v in filtered_vpass.items():
            dot_target_dic=filtered_vpass.copy()
            dot_target_dic.pop(k)
            # print(dot_target_dic)
            mu[k] = v_factor[j].copy()
            # print(mu[k])
            for _,conv_target in dot_target_dic.items():
                mu[k] = mu[k]*conv_target
            # print(mu[k])
            mu[k] = mu[k]/np.sum(mu[k])
            # print(mu[k])
    return mu




In [12]:

def prob_region_sp_decoder(v_factor):
    def print_msg(msg):
        for i in range(4):
            for j in range(6):
                if (i,j) in msg:
                    print(f"msg[{i},{j}]={msg[(i,j)]}",end=" ")
            print("")

    H=np.array(
        [
            [1,1,0,0,1,0],
            [1,1,1,0,0,1],
            [1,0,1,1,0,1],
            [0,0,0,1,1,1]
        ]
    )

    v_pass=dict()
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            if H[i,j] == 1:
                v_pass[(i,j)] = np.array([0.5,0.5])

    v_pass=v_pass.copy()
    for t in range(3):
        print(f"t={t}")
        mu=process_variable_node(v_factor,v_pass)
        print("mu")
        print_msg(mu)
        v_pass=process_check_node(mu)
        print("v_pass")
        print_msg(v_pass)
        print(process_judge(v_factor,v_pass,H))


In [13]:
v_factor=np.array([[1,0],[1,0],[1,0],[0.5,0.5],[0.5,0.5],[0.5,0.5],[0.5,0.5]])
prob_region_sp_decoder(v_factor)

t=0
mu
msg[0,0]=[1. 0.] msg[0,1]=[1. 0.] msg[0,4]=[0.5 0.5] 
msg[1,0]=[1. 0.] msg[1,1]=[1. 0.] msg[1,2]=[1. 0.] msg[1,5]=[0.5 0.5] 
msg[2,0]=[1. 0.] msg[2,2]=[1. 0.] msg[2,3]=[0.5 0.5] msg[2,5]=[0.5 0.5] 
msg[3,3]=[0.5 0.5] msg[3,4]=[0.5 0.5] msg[3,5]=[0.5 0.5] 
v_pass
msg[0,0]=[0.5 0.5] msg[0,1]=[0.5 0.5] msg[0,4]=[1. 0.] 
msg[1,0]=[0.5 0.5] msg[1,1]=[0.5 0.5] msg[1,2]=[0.5 0.5] msg[1,5]=[1. 0.] 
msg[2,0]=[0.5 0.5] msg[2,2]=[0.5 0.5] msg[2,3]=[0.5 0.5] msg[2,5]=[0.5 0.5] 
msg[3,3]=[0.5 0.5] msg[3,4]=[0.5 0.5] msg[3,5]=[0.5 0.5] 
[[0.125 0.   ]
 [0.25  0.   ]
 [0.25  0.   ]
 [0.125 0.125]
 [0.25  0.   ]
 [0.125 0.   ]]
t=1
mu
msg[0,0]=[1. 0.] msg[0,1]=[1. 0.] msg[0,4]=[0.5 0.5] 
msg[1,0]=[1. 0.] msg[1,1]=[1. 0.] msg[1,2]=[1. 0.] msg[1,5]=[0.5 0.5] 
msg[2,0]=[1. 0.] msg[2,2]=[1. 0.] msg[2,3]=[0.5 0.5] msg[2,5]=[1. 0.] 
msg[3,3]=[0.5 0.5] msg[3,4]=[1. 0.] msg[3,5]=[1. 0.] 
v_pass
msg[0,0]=[0.5 0.5] msg[0,1]=[0.5 0.5] msg[0,4]=[1. 0.] 
msg[1,0]=[0.5 0.5] msg[1,1]=[0.5 0.5] msg[1,2]=[0.5 0

In [14]:
v_factor=np.array([[0.75,0.25],[0.75,0.25],[0.75,0.25],[0.75,0.25],[0.25,0.75],[0.75,0.25]])
prob_region_sp_decoder(v_factor)

t=0
mu
msg[0,0]=[0.75 0.25] msg[0,1]=[0.75 0.25] msg[0,4]=[0.25 0.75] 
msg[1,0]=[0.75 0.25] msg[1,1]=[0.75 0.25] msg[1,2]=[0.75 0.25] msg[1,5]=[0.75 0.25] 
msg[2,0]=[0.75 0.25] msg[2,2]=[0.75 0.25] msg[2,3]=[0.75 0.25] msg[2,5]=[0.75 0.25] 
msg[3,3]=[0.75 0.25] msg[3,4]=[0.25 0.75] msg[3,5]=[0.75 0.25] 
v_pass
msg[0,0]=[0.375 0.625] msg[0,1]=[0.375 0.625] msg[0,4]=[0.625 0.375] 
msg[1,0]=[0.5625 0.4375] msg[1,1]=[0.5625 0.4375] msg[1,2]=[0.5625 0.4375] msg[1,5]=[0.5625 0.4375] 
msg[2,0]=[0.5625 0.4375] msg[2,2]=[0.5625 0.4375] msg[2,3]=[0.5625 0.4375] msg[2,5]=[0.5625 0.4375] 
msg[3,3]=[0.375 0.625] msg[3,4]=[0.625 0.375] msg[3,5]=[0.375 0.625] 
No equal
[[0.08898926 0.02990723]
 [0.15820312 0.06835938]
 [0.23730469 0.04785156]
 [0.15820312 0.06835938]
 [0.09765625 0.10546875]
 [0.08898926 0.02990723]]
t=1
mu
msg[0,0]=[0.83219178 0.16780822] msg[0,1]=[0.79411765 0.20588235] msg[0,4]=[0.35714286 0.64285714] 
msg[1,0]=[0.69827586 0.30172414] msg[1,1]=[0.64285714 0.35714286] msg[1,2]=[0.7